In [1]:
from src import feat, mod
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime

today = feat.today_str()

## Model functions

In [2]:
# SRC - mod.py 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MinMaxScaler
import numpy as np
import os
import pandas as pd
import pickle
from xgboost import XGBRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from lightgbm import LGBMRegressor, Dataset
from sklearn.ensemble import RandomForestRegressor
#from sklearn.linear_model import ElasticNet

# Setting the global seed
np.random.seed(10)

# Just easier function to reset indices
def split(data,test_size=1200,random_state=10):
    train, test = train_test_split(data,test_size=test_size,random_state=random_state)
    train = train.reset_index(drop=True)
    test = test.reset_index(drop=True)
    return train, test

def split_weight(data,test_size=1200,weight='pred_split',random_state=10):
    test = data.sample(test_size,weights='split_pred', random_state=random_state)
    train = data[~data['uid'].isin(test['uid'])].reset_index(drop=True)
    test.reset_index(drop=True,inplace=True)
    return train, test

# Just a wrapper around sklearn
# so it returns a pandas dataframe with named
# columns
class DumEnc():
    def __init__(self,dtype=int):
        self.OHE = OneHotEncoder(dtype=dtype,
                                 handle_unknown='ignore',
                                 sparse=False)
        self.var_names = None
    def fit(self, X):
        self.OHE.fit(X)
        cats = self.OHE.categories_
        var = list(X)
        vn = []
        for v,ca in zip(var,cats):
            for c in ca:
                vn.append(f'{v}_{c}')
        self.var_names = vn
    def transform(self,X):
        res = pd.DataFrame(self.OHE.transform(X),
                           columns=self.var_names,
                           index=X.index)
        return res

def dummy_stats(values,begin_date):
    vdate = pd.to_datetime(values,errors='ignore')
    year = vdate.dt.year
    month = vdate.dt.month
    week_day = vdate.dt.dayofweek
    diff_days = (vdate - begin_date).dt.days
    # if binary, turn week/month into dummy variables
    return diff_days, week_day, month, year

def circle_stats(values,begin_date):
    vdate = pd.to_datetime(values,errors='ignore')
    within_year = vdate.dt.dayofyear
    week_day = vdate.dt.dayofweek
    # calculate sine/cosine for within year
    year_cos = np.cos(within_year*(2*np.pi/365))
    year_sin = np.sin(within_year*(2*np.pi/365))
    # calculate sine/cosine for within week
    week_cos = np.cos(week_day*(2*np.pi/7))
    week_sin = np.sin(week_day*(2*np.pi/7))
    diff_days = (vdate - begin_date).dt.days
    return diff_days, year_cos, year_sin, week_cos, week_sin


class DateEnc():
    def __init__(self,
                 begin = '1/1/2015',
                 dummy = True,
                 dum_types=['days','weekday','month']):
        self.begin = pd.to_datetime(begin)
        self.dummy = dummy
        # 'days','weekday','month','year'
        self.dum_types = dum_types
        self.cat_vars = []
        # Setting categorical variables
    def fit(self,X):
        # These are just fixed functions
        pass
    def transform(self,X):
        vars = list(X)
        res = []
        res_labs = []
        cat_labs = []
        if self.dummy:
            for v in vars:
                dd, week_day, month, year = dummy_stats(X[v],self.begin)
                if 'days' in self.dum_types:
                    res.append(dd) # this is not likely to be categorical
                    res_labs.append(f'days_{v}')
                if 'weekday' in self.dum_types:
                    res.append(week_day)
                    res_labs.append(f'weekday_{v}')
                    cat_labs.append(f'weekday_{v}')
                if 'month' in self.dum_types:
                    res.append(month)
                    res_labs.append(f'month_{v}')
                    cat_labs.append(f'weekday_{v}')
                if 'year' in self.dum_types:
                    res.append(year)
                    res_labs.append(f'year_{v}')
                    cat_labs.append(f'year_{v}')
            self.cat_vars = cat_labs
        else:
             for v in vars:
                dd, year_cos, year_sin, week_cos, week_sin = circle_stats(X[v],self.begin)
                res += [dd, year_cos, year_sin, week_cos, week_sin]
                res_labs += [f'days_{v}',f'yearcos_{v}',f'yearsin_{v}',f'weekcos_{v}',f'weeksin_{v}']
        res_df = pd.concat(res,axis=1)
        res_df.index = X.index
        res_df.columns = res_labs
        return res_df


# Spline encoding
# defaults to regular knots
# or specified locations
class SplEnc():
    def __init__(self):
        pass
    def fit(self,X):
        pass
    def transform(self,X):
        pass


class IdentEnc():
    def __init__(self):
        self.note = None
    def fit(self,X):
        pass
    def transform(self,X):
        return X.copy()


class SimpleOrdEnc():
    def __init__(self,
                 dtype=int,
                 unknown_value=-1,
                 lim_k=None,
                 lim_count=None):
        self.unknown_value = unknown_value
        self.dtype = dtype
        self.lim_k = lim_k
        self.lim_count = lim_count
        self.vars = None
        self.soe = None
    def fit(self, X):
        self.vars = list(X)
        # Now creating fit for each variable
        res_oe = {}
        for v in list(X):
            res_oe[v] = OrdinalEncoder(dtype=self.dtype,
                handle_unknown='use_encoded_value',
                        unknown_value=self.unknown_value)
            # Get unique values minus missing
            xc = X[v].value_counts().reset_index()
            xc.columns = [v, "Freq"]
            # If lim_k, only taking top K value
            if self.lim_k:
                top_k = self.lim_k - 1
                un_vals = xc.loc[0:top_k,:]
            # If count, using that to filter
            elif self.lim_count:
                un_vals = xc[xc["Freq"] >= self.lim_count].copy()
            # If neither
            else:
                un_vals = xc
            # Now fitting the encoder for one variable
            res_oe[v].fit(un_vals[[v]])
        # Appending back to the big class
        self.soe = res_oe
    # Defining transform/inverse_transform classes
    def transform(self, X):
        xcop = X[self.vars].copy()
        for v in self.vars:
            xcop[v] = self.soe[v].transform( X[[v]].fillna(self.unknown_value) )
        return xcop
    def fit_transform(self,X):
        self.fit(X)
        return self.transform(X)
    def inverse_transform(self, X):
        xcop = X[self.vars].copy()
        for v in self.vars:
            xcop[v] = self.soe[v].inverse_transform( X[[v]].fillna(self.unknown_value) )
        return xcop

# You can compose multiple feature engines together
# Such as DateEnc() and DumEnc()
class ComposeFE():
    def __init__(self,
                mods):
        # expects a dictionary
        # of models
        self.mods = mods
    def fit(self,X):
        trans = X.copy()
        # goes left to right
        for k,m in self.mods.items():
            m.fit(trans)
            trans = m.transform(trans)
    def transform(self,X):
        trans = X.copy()
        for k,m in self.mods.items():
            trans = m.transform(trans)
        return trans


class FeatureEngine():
    def __init__(self,
                 ord_vars = None,
                 dum_vars = None,
                 spl_vars = None,
                 dat_vars = None,
                 ide_vars = None,
                 scale = None):
        self.fin_vars = None
        self.enc_dict = {}
        self.ord_vars = ord_vars
        self.dum_vars = dum_vars
        self.spl_vars = spl_vars
        self.dat_vars = dat_vars
        self.ide_vars = ide_vars
        self.cat_vars = None
        self.ord = SimpleOrdEnc()
        self.dum = DumEnc()
        self.dat = DateEnc()
        self.spl = SplEnc()
        self.ide = IdentEnc()
        self.scale = scale
        enc_vars = [ord_vars,dum_vars,spl_vars,dat_vars,ide_vars]
        enc_mods = [self.ord, self.dum, self.spl, self.dat, self.ide]
        for v,m in zip(enc_vars,enc_mods):
            if v is not None:
                self.enc_dict[tuple(v)] = m
    def fit(self, X):
        res = []
        for v,m in self.enc_dict.items():
            m.fit(X[list(v)])
            rf = m.transform(X[list(v)])
            res.append(rf)
        # doing a transform to know the variable names in the end
        res_df = pd.concat(res,axis=1)
        if self.scale is not None:
            self.scale.fit(res_df)
            res_df = pd.DataFrame(self.scale.transform(res_df),columns=list(res_df))
        self.fin_vars = list(res_df)
        # Adding categorical variables back in
        cat_vars = []
        if self.dum_vars is not None:
            cat_vars += self.dum.var_names
        if self.dat_vars is not None:
            cat_vars += self.dat.cat_vars
        if self.ord_vars is not None:
            cat_vars += self.ord_vars
        self.cat_vars = cat_vars
        return res_df
    def transform(self, X):
        res = []
        for v,m in self.enc_dict.items():
            res.append(m.transform(X[list(v)]))
        res_df = pd.concat(res,axis=1)
        if self.scale is not None:
            res_df = pd.DataFrame(self.scale.transform(res_df),columns=list(res_df))
        return res_df

def safelog(x):
    return np.log(x.clip(1))

def strat(values):
    edges = [np.NINF,20000,1e6,1e7,1e8,np.inf]
    labs = [1,2,3,4,5]
    res = pd.cut(values,bins=edges,labels=labs,right=False)
    return res.astype(int)


# Class to insert different types of
# regression models
class RegMod():
    def __init__(self,
                 ord_vars = None,
                 dum_vars = None,
                 dat_vars = None,
                 ide_vars = None,
                        y = None,
                transform = None,
                inv_trans = None,
                   weight = None,
                  scale_x = None,
                      mod = XGBRegressor(n_estimators=100, max_depth=3)):
        self.fe = FeatureEngine(ord_vars=ord_vars,
                           dat_vars=dat_vars,
                           dum_vars=dum_vars,
                           ide_vars=ide_vars,
                           scale = scale_x)
        self.transform = transform
        self.inv_trans = inv_trans
        self.mod = mod
        self.y = y
        self.resids = None
        self.cat_vars = None
        self.weight = weight
        self.metrics = None
        self.fit_cat = False
    def fit(self, X, weight=True, cat=True):
        if (self.weight is not None) & weight:
            sw = X[self.weight]
        else:
            sw = None
            #print('NOT using Weights in fit')
        y_dat = X[self.y].copy()
        if self.transform:
            y_dat = self.transform(y_dat)
        X_dat = self.fe.fit(X)
        self.cat_vars = self.fe.cat_vars
        # If catboost or lightgbm, pass in categories
        if (type(self.mod) == CatBoostRegressor) & cat:
            vt = list(X_dat)
            if self.cat_vars is not None:
                ci = [vt.index(c) for c in self.cat_vars]
            else:
                ci = None
            self.mod.fit(X_dat,y_dat,sample_weight=sw,cat_features=ci)
            self.fit_cat = True
        elif (type(self.mod) == LGBMRegressor) & cat:
            self.fit_cat = True
            for v in self.cat_vars:
                X_dat[v] = X_dat[v].astype('category')
            self.mod.fit(X_dat, y_dat, sample_weight=sw)
        else:
            self.mod.fit(X_dat,y_dat,sample_weight=sw)
        pred = self.mod.predict(X_dat)
        self.resids = pd.Series(y_dat - pred)
    def predict(self,X,duan=True):
        X_dat = self.fe.transform(X)
        if self.fit_cat & (type(self.mod) == LGBMRegressor):
            for v in self.cat_vars:
                X_dat[v] = X_dat[v].astype('category')
        pred = pd.Series(self.mod.predict(X_dat), X.index)
        resids = self.resids
        # if transform, do Duans smearing
        if (self.transform is not None) & duan:
            resids = resids.values.reshape(1,resids.shape[0])
            dp = self.inv_trans(pred.values.reshape(X.shape[0],1) + resids)
            pred = pd.Series(dp.mean(axis=1), X.index)
        return pred
    def predict_int(self,X):
        pred = self.predict(X)
        if self.transform:
            pred = strat(pred)
        pred = pred.clip(1,5).round().astype(int)
        return pred
    def feat_import(self):
        var_li = self.fe.fin_vars
        mod_fi = self.mod.feature_importances_
        res_df = pd.DataFrame(zip(var_li,mod_fi),columns = ['Var','FI'])
        res_df.sort_values('FI',ascending=False,inplace=True,ignore_index=True)
        # Normalize to sum to 1
        res_df['FI'] = res_df['FI']/res_df['FI'].sum()
        return res_df
    def met_eval(self, data, weight=True, cat=True, full_train=False,
                 split_tt='weighted', test_size=2000, test_splits=10, 
                 ret=False, pr=False):
        dc = data.copy()
        seeds = np.random.randint(1,1e6,test_splits)
        metrics = []
        for s in seeds:
            if split_tt == 'weighted':
                if self.weight is not None:
                    wv = self.weight
                else:
                    wv = 'pred_split'
                train, test = split_weight(dc,test_size,wv,s)
            else:
                train, test = split(dc,test_size,s)
            self.fit(train, weight, cat)
            test['pred'] = self.predict_int(test)
            met_di = rmse_region(test,ret=True)
            met_di['seed'] = s
            metrics.append(met_di.copy())
        mpd = pd.DataFrame(metrics)
        if full_train:
            self.fit(data)
        if self.metrics is None:
            self.metrics = mpd
        else:
            self.metrics = pd.concat([self.metrics,mpd],axis=0)
        if pr:
            print(mpd[['AvgError','midwest','northeast','south','west']].describe().T)
        if ret:
            return mpd['AvgError'].mean()

class CatMod():
    def __init__(self,
                 ord_vars = None,
                 dum_vars = None,
                 dat_vars = None,
                 ide_vars = None,
                        y = None,
                transform = None,
                inv_trans = None,
                  scale_x = None,
                      mod = CatBoostClassifier(iterations=100,depth=5,allow_writing_files=False,verbose=False)
                      ):
        self.fe = FeatureEngine(ord_vars=ord_vars,
                           dat_vars=dat_vars,
                           dum_vars=dum_vars,
                           ide_vars=ide_vars,
                           scale = scale_x)
        self.mod = mod
        self.y = y
        self.cat_vars = None
    def fit(self, X, sample_weight=None):
        y_dat = X[self.y].copy().astype(int)
        X_dat = self.fe.fit(X)
        self.mod.fit(X_dat,y_dat,sample_weight=sample_weight)
    def predict_proba(self,X):
        X_dat = self.fe.transform(X)
        pred_probs = self.mod.predict_proba(X_dat)
        # Turning into nicer dataframe
        cols = [f'P{str(i)}' for i in range(pred_probs.shape[1])]
        pred_probs = pd.DataFrame(pred_probs,index=X.index, columns=cols)
        return pred_probs
    def predict(self,X):
        # returns predicted probability
        pred = self.predict_proba(X)
        return pred["P1"]
    def feat_import(self):
        var_li = self.fe.fin_vars
        mod_fi = self.mod.feature_importances_
        res_df = pd.DataFrame(zip(var_li,mod_fi),columns = ['Var','FI'])
        res_df.sort_values('FI',ascending=False,inplace=True,ignore_index=True)
        # Normalize to sum to 1
        res_df['FI'] = res_df['FI']/res_df['FI'].sum()
        return res_df

# If you pass in multiple models
# this will ensemble them, presumes regressor models
class EnsMod():
    def __init__(self, mods, av_func = 'mean'):
        self.mods = mods #should be dict
        self.av_func = av_func
    def fit(self,X,weight=True,cat=True):
        for key,mod in self.mods.items():
            mod.fit(X,weight=weight,cat=cat)
    def predict(self,X):
        res = []
        for key,mod in self.mods.items():
            res.append(mod.predict(X))
        res_df = pd.concat(res,axis=1)
        if self.av_func == 'mean':
            pred = res_df.mean(axis=1)
        return pred
    def predict_int(self,X):
        pred = self.predict(X)
        pred = pred.clip(1,5).round().astype(int)
        return pred

def rmse_region(data,pred='pred',true='severity',region='region',scale=False,ret=False):
    dc = data[[region,true,pred]].copy()
    if scale:
        dc[pred] = strat(dc[pred])
    dc[pred] = dc[pred].round().astype(int).clip(1,5)
    dc[region] = dc[region].replace({1:'northeast',
                                     2:'south',
                                     3:'midwest',
                                     4:'west'})
    dc['sq_error'] = (dc[true] - dc[pred])**2
    gr_val = dc.groupby(region,as_index=False)['sq_error'].mean()
    gr_val['root_mse'] = np.sqrt(gr_val['sq_error'])
    avg_error = gr_val['root_mse'].mean()
    if ret:
        regions = gr_val['region'].tolist()
        regions.append('AvgError')
        vals = gr_val['root_mse'].tolist()
        vals.append(avg_error)
        gr_di = {r:v for r,v in zip(regions,vals)}
        return gr_di
    else:
        print(f'\nAverage error {avg_error:.4f}')
        print('\nRegion Error')
        print(gr_val[['region','root_mse']])

def save_model(mod,name):
    fname = f'./models/{name}.pkl'
    outfile = open(fname,"wb")
    pickle.dump(mod,outfile)
    outfile.close()

def load_model(name):
    fname = f'./models/{name}.pkl'
    infile = open(fname, "rb")
    mod = pickle.load(infile)
    infile.close()
    return mod


# function to check if similar to any past submissions
def check_similar(current):
    files = os.listdir("./submissions")
    for fi in files:
        old = pd.read_csv(f"./submissions/{fi}")
        dif = np.abs(current['severity'] - old['severity']).sum()
        if dif == 0:
            print(f'Date {fi} same as current')


def check_day(current,day="sub_2023_01_31.csv"):
    old = pd.read_csv(fr"./submissions/{day}")
    dstr = f'dif_{day[4:-4]}'
    current[dstr] = old['severity'] - current['severity']
    print(current[dstr].value_counts())

In [3]:
#src - feat.py

def today_str():
    now = datetime.now()
    return now.strftime('%Y_%m_%d')


reg_ord = {'west':4,
           'midwest':3,
           'south':2,
           'northeast':1}

# ordinal encoding for region
def org_reg(data,rstr='region'):
    data[rstr] = data[rstr].replace(reg_ord)

def filter_reg(data, region):
    return data.loc[data['region']==region]

def ord_imtype(data,imstr='imtype'):
    rep_di = {'land_sat':0,
              'sentinel':1}
    data[imstr] = data[imstr].fillna(-1).replace(rep_di)


def filter_landsat(data):
    rep_di = {'land_sat':0,
              'sentinel':1}
    data['imtype'] = data['imtype'].fillna(-1).replace(rep_di)
    im_vars = ['prop_lake_500', 'r_500', 'g_500', 'b_500']
    im_vars += ['prop_lake_1000', 'r_1000', 'g_1000', 'b_1000']
    im_vars += ['prop_lake_2500', 'r_2500', 'g_2500', 'b_2500']
    im_vars += ['imtype']
    landsat = data['imtype'] == 0
    data.loc[landsat,im_vars] = -1

def safesqrt(values):
    return np.sqrt(values.clip(0))

def safelog(x):
    return np.log(x.clip(1))

def strat(values):
    edges = [np.NINF,20000,1e6,1e7,1e8,np.inf]
    labs = [1,2,3,4,5]
    res = pd.cut(values,bins=edges,labels=labs,right=False)
    return res.astype(int)

# Looking at train/test
# there are a few clusters, want
# to make sure to predict these well
def cluster(x):
    lat, lon = x[0], x[1]
    if (lat < 41) & (lon < -116):
        # cali
        return 7
    elif (lat < 41) & (lat > 36.29) & (lon < -92.9) & (lon > -102.2):
        # midwest
        return 6
    elif (lat < 38.14) & (lat > 33.26) & (lon < -74.8) & (lon > -85.52):
        # carolina
        return 2
    elif (lat < 43) & (lat > 38.7) & (lon < -75.4) & (lon > -83.55):
        # erie
        return 3
    elif (lat < 43.1) & (lat > 40.7) & (lon < -69.5) & (lon > -74.6):
        # mass
        return 4
    elif (lat < 49.6) & (lat > 41.5) & (lon < -83.55) & (lon > -104.56):
        # dakota
        return 1
    else:
        # other
        return 5

#                  1   2      2   3    3     4   4    5   5
#te_st = pd.Series([1,20000,30000,1e6,1e6+1,1e7,1e7+1,1e8,1e9])
#print(strat(te_st))

db = './data/data.sqlite'

train_query = """
SELECT 
  m.uid,
  l.region,
  l.severity,
  l.density,
  m.latitude,
  m.longitude,
  m.date,
  e.elevation,
  e.mine,
  e.maxe,
  e.dife,
  e.avge,
  e.stde,
  sl.severity_100,
  sl.logDensity_100,
  sl.count_100,
  sl.severity_300,
  sl.logDensity_300,
  sl.count_300,
  sl.severity_1000,
  sl.logDensity_1000,
  sl.count_1000,
  st.imtype,
  st.prop_lake_500,
  st.r_500,
  st.g_500,
  st.b_500,
  st.prop_lake_1000,
  st.r_1000,
  st.g_1000,
  st.b_1000,
  st.prop_lake_2500,
  st.r_2500,
  st.g_2500,
  st.b_2500
FROM meta AS m
LEFT JOIN elevation_dem AS e
  ON m.uid = e.uid
LEFT JOIN spat_lag AS sl
  ON m.uid = sl.uid
LEFT JOIN sat AS st
  ON m.uid = st.uid
LEFT JOIN labels AS l
  ON m.uid = l.uid
WHERE
  m.split = 'train'
"""

test_query = """
SELECT 
  m.uid,
  l.region,
  m.latitude,
  m.longitude,
  m.date,
  e.elevation,
  e.mine,
  e.maxe,
  e.dife,
  e.avge,
  e.stde,
  sl.severity_100,
  sl.logDensity_100,
  sl.count_100,
  sl.severity_300,
  sl.logDensity_300,
  sl.count_300,
  sl.severity_1000,
  sl.logDensity_1000,
  sl.count_1000,
  st.imtype,
  st.prop_lake_500,
  st.r_500,
  st.g_500,
  st.b_500,
  st.prop_lake_1000,
  st.r_1000,
  st.g_1000,
  st.b_1000,
  st.prop_lake_2500,
  st.r_2500,
  st.g_2500,
  st.b_2500
FROM meta AS m
LEFT JOIN elevation_dem AS e
  ON m.uid = e.uid
LEFT JOIN spat_lag AS sl
  ON m.uid = sl.uid
LEFT JOIN sat AS st
  ON m.uid = st.uid
LEFT JOIN format AS l
  ON m.uid = l.uid
WHERE
  m.split = 'test'
"""

def add_table(data,tab_name,db_str=db):
    db_con = sqlite3.connect(db_str)
    dn = data.copy()
    dn['DateTime'] = pd.to_datetime('now',utc=True)
    dn.to_sql(tab_name,index=False,if_exists='replace',con=db_con)


def get_both(db_str=db,split_pred=False):
    r1 = get_data('train',db_str,split_pred)
    r1['test'] = 0
    r1.drop(columns=['severity','density','logDensity'],inplace=True)
    r2 = get_data('test',db_str,split_pred)
    r2['test'] = 1
    res_df = pd.concat([r1,r2],axis=0)
    return res_df.reset_index(drop=True)

def get_data(region, data_type='train',db_str=db,split_pred=False):
    db_con = sqlite3.connect(db_str)
    if data_type == 'train':
        sql = train_query
    elif data_type == 'test':
        sql = test_query
    dat = pd.read_sql(sql,con=db_con)
    org_reg(dat) # Region ordinal encode
    # Winning solution used landsat-7 data
    #ord_imtype(dat) # image type landsat/sentinel
    filter_landsat(dat) # filtering mistake landsat-7 info
    dat = dat.fillna(-1) # missing a bit of sat data
    dat['cluster'] = dat[['latitude','longitude']].apply(cluster,axis=1)
    dat = filter_reg(dat, region)
    if data_type == 'train':
        dat['logDensity'] = safelog(dat['density'])
    if split_pred:
        pred_test = pd.read_sql('SELECT uid, pred AS split_pred FROM split_pred',con=db_con)
        dat = dat.merge(pred_test,on='uid')
    return dat


# Need logic to take predictions and get them in the right order
def sub_format(data,pred='pred'):
    form = pd.read_csv('./data/submission_format.csv')
    # some logic to transform predictions via Duan
    # smearing
    dp = data[[pred,'uid']].copy()
    dp[pred] = dp[pred].round().astype(int).clip(1,5)
    mf = form.merge(dp,on='uid')
    mf['severity'] = mf['pred']
    return mf[['uid','region','severity']]

In [6]:
#src - get_data.py
'''
Functions to get different data sources
and cache them in sqllite DB
'''

import requests
import numpy as np
import pandas as pd
import sqlite3
from io import StringIO
import time
import os
from datetime import datetime, timedelta
from math import sin, cos, tan, acos, atan, atan2, radians, sqrt, pi
import rioxarray
import planetary_computer as pc
import geopy.distance as distance
import odc.stac
#import cv2


db = './data/data.sqlite'
db_con = sqlite3.connect(db)

# having a hard time installing opencv on
# planetary computer, only use this function
# so far

def cv2_norm(matrix):
    amin = matrix.min()
    amax = matrix.max()
    modif = amax/(amax - amin)
    shift_mat = np.round((matrix - amin)*modif)
    return shift_mat.astype(int)


# chunking pandas dataframe
def chunk_pd(df, chunk_size):
    nrows = df.shape[0]
    tot_split = np.ceil(nrows/chunk_size)
    split_rows = np.array_split(np.arange(nrows), tot_split)
    fin_list_df = [df.iloc[s, :] for s in split_rows]
    return fin_list_df

# Get table names
def get_table_names(con=db_con):
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    res = pd.read_sql(query,con)
    return res['name'].tolist()

# Drop table if needed
def drop_table(table,con=db_con):
    dt = f'DROP TABLE {table}'
    con.execute(dt)

# Check if table exists
def tab_exists(table,con=db_con):
    rt = get_table_names(con)
    res = table in rt
    return res

# Add a table with DateTime appended
def add_table(data,tab_name,con=db_con):
    dn = data.copy()
    dn['DateTime'] = pd.to_datetime('now',utc=True)
    dn.to_sql(tab_name,index=False,if_exists='append',con=con)

# Seeing to update data based on old table
def get_update(data,table_name,con=db_con):
    query = f'SELECT DISTINCT uid FROM {table_name};'
    uids = pd.read_sql(query,con=con)['uid'].tolist()
    notin_old = ~data['uid'].isin(uids)
    return data[notin_old].copy()

# Adds in competition meta data
def add_meta_data(con=db_con):
    ret_dat = {}
    name_map = {'meta': './data/metadata.csv',
                'labels': './data/train_labels.csv',
                'format': './data/submission_format.csv'}
    for n,l in name_map.items():
        if tab_exists(n):
            query = f'SELECT * from {n}'
            d = pd.read_sql(query,con=con)
            ret_dat[n] = d
        else:
            d = pd.read_csv(l)
            add_table(d,n,con)
            ret_dat[n] = d
    return ret_dat

# Just to make sure those tables are populated
meta_data = add_meta_data()

###########################################################################################
# VINCENTY FORMULA FOR DISTANCE BETWEEN LAT/LON

# The function below is  based on transcribing and adapting code from
# http://www.movable-type.co.uk/scripts/latlong-vincenty.html
# which includes the note
#I offer these formulae & scripts for free use and adaptation as my contribution to the open-source 
#info-sphere from which I have received so much. You are welcome to re-use these scripts 
#[under a simple attribution license, without any warranty express or implied] 
#provided solely that you retain my copyright notice and a link to this page.[above]
#If you have any queries or find any problems, contact me at ku.oc.epyt-elbavom@oeg-stpircs.
#
#(c) 2002-2011 Chris Veness 

#Another useful source is
#http://search.cpan.org/~bluefeet/GIS-Distance-0.01001/lib/GIS/Distance/Vincenty.pm

# ellipsoid parameters
class Elp:
    a = 6378137.
    b = 6356752.314245
    f = (a-b) / a

def ellipseDist(lat1, lon1, lat2, lon2, inradians=False):
    """Return the distance in meters between the two sets of coordinates  using an ellipsoidal approximation to the earth 
    
    Calculation uses Vincenty inverse ellipsoid formula with WGS-84 ellipsoid parameters.
    
    lat1 and lon1 are the latitude and longitude of the first point and lat2, lon2 of the second.
    if inradians is True, locations are in radians; otherwise they are in degrees.
    If the algorithm fails to converge, the return value is None, which will become SYSMIS when returned to SPSS
    
    The ellipsoid model is more accurrate than the simpler spherical approximation."""
    
    if lat1 is None or lon1 is None or lat2 is None or lon2 is None:
        return None
    if not inradians:
        lat1, lon1, lat2, lon2 = [radians(ang) for ang in [lat1, lon1, lat2, lon2]]
    u1, u2 = [atan((1-Elp.f) * tan(x)) for x in [lat1, lat2]]
    sinu1, sinu2 = sin(u1), sin(u2)
    cosu1, cosu2 = cos(u1), cos(u2)
    londif = lon2 - lon1
    
    lam = londif
    lamp = 2 * pi
    numiter = 50  # maximum number of iterations
    
    for i in range(numiter):
        if abs(lam - lamp) <= 1e-12:
            break
        sinLam = sin(lam)
        cosLam = cos(lam)
        sinSigma = sqrt((cosu2 * sinLam) **2 + (cosu1 * sinu2-sinu1 * cosu2 * cosLam)**2)
        if sinSigma == 0:
            return 0
        cosSigma = sinu1 * sinu2 + cosu1 * cosu2 * cosLam
        sigma = atan2(sinSigma, cosSigma)
        sinAlpha = cosu1 * cosu2 * sinLam / sinSigma
        cosSqAlpha = 1 - sinAlpha **2
        try:
            cos2SigmaM = cosSigma - 2 * sinu1 * sinu2 / cosSqAlpha
        except:
            cos2SigmaM = 0 # equatorial line
        c = (Elp.f/16) * cosSqAlpha * (4 + Elp.f * (4 - 3 * cosSqAlpha))
        lamp = lam
        lam = londif + (1 - c)  * Elp.f * sinAlpha\
            * (sigma + c * sinSigma* (cos2SigmaM + c * cosSigma * (-1 + 2 * cos2SigmaM **2)))
    else:
            return None
        
    uSq = cosSqAlpha * (Elp.a * Elp.a - Elp.b * Elp.b) / (Elp.b * Elp.b)
    A = 1 + uSq / 16384 * (4096+uSq * (-768 + uSq * (320-175 * uSq)))
    B = uSq/1024 * (256 + uSq * (-128 + uSq * (74-47 * uSq)))
    deltaSigma = B * sinSigma * (cos2SigmaM + B / 4 * (cosSigma * (-1+2 * cos2SigmaM **2)- \
        B / 6 * cos2SigmaM * (-3 + 4 * sinSigma **2) * (-3 + 4 * cos2SigmaM **2)))
    s = Elp.b * A * (sigma - deltaSigma)
    return s

# need to apply this over a vector, makes it ok with pandas dataframe
# returns in kilometers now
def vector_vincent(x, lat2, lon2):
    return ellipseDist(x[0], x[1], lat2, lon2, inradians=False)/1000

#lat = [35,36,37,38]
#lon = [-72,-71,-72,-71]
#test_df = pd.DataFrame(zip(lat,lon),columns=['lat','lon'])
#dist = test_df.apply(vector_vincent,args=(35.1,-72.1),axis=1)

###########################################################################################


# Average outcome in data from sites within d kilometers (default 300)
# Limit to only those before outcome
def spatial_lag(pred_data,data,distance=[100,300,1000],fields=['severity','logDensity'],lat='latitude',lon='longitude',date='date'):
    res = [] # final list with info
    # prepping data to check
    dc = data[fields + [lat,lon,date]].copy()
    dc[date] = pd.to_datetime(dc[date])
    # now prepping pred_data
    plon_li = pred_data[lon].tolist()
    plat_li = pred_data[lat].tolist()
    pdat_li = pd.to_datetime(pred_data[date]).tolist()
    for plon,plat,pdat in zip(plon_li,plat_li,pdat_li):
        loc_dat = []
        count_dat = []
        sub_dc = dc[dc[date] < pdat].copy()
        km_dist = sub_dc[[lat,lon]].apply(vector_vincent,args=(plat,plon),axis=1)
        for d in distance:
            sub_dist = sub_dc[km_dist < d].copy()
            count_dat.append(sub_dist.shape[0])
            for v in fields:
                loc_dat.append(sub_dist[v].mean())
        res.append(loc_dat + count_dat)
    res_labs = []
    for d in distance:
        for v in fields:
            res_labs.append(f'{v}_{str(d)}')
    res_labs += [f'count_{str(d)}' for d in distance]
    res_pd = pd.DataFrame(res,columns=res_labs,index=pred_data.index)
    return res_pd.fillna(-1)


def get_spatiallag(con=db_con,table_name='spat_lag'):
    # get full metadata table
    full_dat = pd.read_csv('./data/metadata.csv')
    # get metadata with only labels
    labs = pd.read_csv('./data/train_labels.csv')
    only_lab = labs.merge(full_dat,on='uid')
    only_lab['logDensity'] = np.log(only_lab['density'].clip(1))
    # create spatial lag
    lag_df = spatial_lag(full_dat,only_lab)
    lag_df['uid'] = full_dat['uid']
    # save to a new table
    add_table(lag_df,table_name)
    return lag_df


#########################################################################
# helper functions from DataDriven post
# https://drivendata.co/blog/tick-tick-bloom-benchmark


def get_bounding_box(latitude, longitude, meter_buffer=1000):
    """
    Given a latitude, longitude, and buffer in meters, returns a bounding
    box around the point with the buffer on the left, right, top, and bottom.
    Returns a list of [minx, miny, maxx, maxy]
    """
    distance_search = distance.distance(meters=meter_buffer)
    # calculate the lat/long bounds based on ground distance
    # bearings are cardinal directions to move (south, west, north, and east)
    min_lat = distance_search.destination((latitude, longitude), bearing=180)[0]
    min_long = distance_search.destination((latitude, longitude), bearing=270)[1]
    max_lat = distance_search.destination((latitude, longitude), bearing=0)[0]
    max_long = distance_search.destination((latitude, longitude), bearing=90)[1]
    return [min_long, min_lat, max_long, max_lat]


# get our date range to search, and format correctly for query
def get_date_range(date, time_buffer_days=15):
    """Get a date range to search for in the planetary computer based
    on a sample's date. The time range will include the sample date
    and time_buffer_days days prior
    Returns a string"""
    datetime_format = "%Y-%m-%dT"
    range_start = pd.to_datetime(date) - timedelta(days=time_buffer_days)
    date_range = f"{range_start.strftime(datetime_format)}/{pd.to_datetime(date).strftime(datetime_format)}"
    return date_range


def crop_sentinel_image(item, bounding_box):
    """
    Given a STAC item from Sentinel-2 and a bounding box tuple in the format
    (minx, miny, maxx, maxy), return a cropped portion of the item's visual
    imagery in the bounding box.
    Returns the image as a numpy array with dimensions (color band, height, width)
    """
    (minx, miny, maxx, maxy) = bounding_box

    image = rioxarray.open_rasterio(pc.sign(item.assets["visual"].href)).rio.clip_box(
        minx=minx,
        miny=miny,
        maxx=maxx,
        maxy=maxy,
        crs="EPSG:4326",
    )
    # Should I return X/Y as well?
    return cv2_norm(image.to_numpy())


def crop_landsat_image(item, bounding_box):
    """
    Given a STAC item from Landsat and a bounding box tuple in the format
    (minx, miny, maxx, maxy), return a cropped portion of the item's visual
    imagery in the bounding box.
    Returns the image as a numpy array with dimensions (color band, height, width)
    """
    (minx, miny, maxx, maxy) = bounding_box

    image = odc.stac.stac_load(
        [pc.sign(item)], bands=["red", "green", "blue"], bbox=[minx, miny, maxx, maxy]
    ).isel(time=0)
    image_array = image[["red", "green", "blue"]].to_array().to_numpy()

    # normalize to 0 - 255 values
    #image_array = cv2.normalize(image_array, None, 0, 255, cv2.NORM_MINMAX)
    image_array = cv2_norm(image_array)

    return image_array

In [8]:
# main_prepdata.py
'''
So you can either do it two ways
One is to run each of the files
individually, e.g.
cd ./algaebloom
python get_dem.py
python get_lag.py
python get_sat.py
These take along time, and dem/sat can
fail periodically
They are not idempotent, so incrementally save 
the values to a SQLite database (in the data folder)
The easier way though to replicate the SQLite database
though is to run this script, it presumes you have the files
algaebloom/data
       - metadata.csv           # from competition
       - train_labels.csv       # from competition
       - submission_format.csv  # from competition
       - elevation_dem.csv      # generated via get_dem.py
       - spat_lag.csv           # generated via get_lag.py
       - sat.csv                # generated via get_sat.py
       - split_pred.csv         # generated via get_split.py
The final solution does not use the lag values, but have
included to replicate my prior tests. Get split needs to be
run AFTER the elevation stats are prepped, but otherwise 
the order does not matter
any questions? Feel free to email me, 
apwheele@gmail.com
Andy Wheeler
'''


import pandas as pd
import sqlite3

db = './data/data.sqlite'
tab_names = ['elevation_dem','split_pred','spat_lag','sat']
meta_names = ['meta','labels', 'format']
meta_csv = ['metadata','train_labels','submission_format']

fd = {c:t for c,t in zip(meta_csv,meta_names)}
ft = {t:t for t in tab_names}
fd.update(ft)

# Function to save out csv files
def save_csv(db=db):
    db_con = sqlite3.connect(db)
    for t in tab_names:
        res = pd.read_sql(f'SELECT * FROM {t}',db_con)
        res.to_csv(f'./data/{t}.csv',index=False)

# Function to prep SQLite DB
def prep_sql(db=db):
    db_con = sqlite3.connect(db)
    for csv,tab_name in fd.items():
        res = pd.read_csv(f'./data/{csv}.csv')
        res.to_sql(tab_name,index=False,if_exists='replace',con=db_con)
    # Showing resulting table names
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    rt = pd.read_sql(query,db_con)
    print('Resulting table names in sqlite')
    print(rt['name'].tolist())

if __name__ == "__main__":
    print('Executing script to prep data in sqlite')
    print('This expects the csv files listed at front')
    print('of script to be in the data folder')
    prep_sql()

Executing script to prep data in sqlite
This expects the csv files listed at front
of script to be in the data folder
Resulting table names in sqlite
['meta', 'labels', 'format', 'elevation_dem', 'split_pred', 'spat_lag', 'sat']


## Tuning

In [24]:
import optuna

res_results = {}

# Setting up data and variable sets

train_dat = feat.get_data(split_pred=True)

In [25]:
# Model fit with weights
weight_cats = tuple([False])

# Model fit with categorical variables
cat_cats = (True,False)

# Lat/Lon included in model
xy_cats = {'no': [],
           'both': ['latitude','longitude'],
           'lat': ['latitude'],
           'lon': ['longitude']}

xy_keys = tuple(xy_cats.keys())

# Region variables
region_cats = {'both': ['region','cluster'],
               'reg':  ['region'],
               'clust':  ['cluster']}

reg_keys = tuple(region_cats.keys())

# Elevation Variables
ele_cats = {'max_dif':['maxe','dife'],
            'all_var':['maxe','dife','elevation','stde'],
            'ele_std':['elevation','stde'],
            'ele_dif':['elevation','dife'],
            'max_std':['maxe','stde'] }

ele_keys = tuple(ele_cats.keys())

In [26]:
# Spatial Lag Variables
sl_cats = {#'lag100': ['severity_100','logDensity_100','count_100'],
           #'lag300': ['severity_300','logDensity_300','count_300'],
           #'lag1000': ['severity_1000','logDensity_1000','count_1000'],
           'lagNone': []}

sl_keys = tuple(sl_cats.keys())

# Sat imagery data
sat_cats = {'sat500': ['imtype','prop_lake_500','r_500','g_500','b_500'],
            'sat1000': ['imtype','prop_lake_1000','r_1000','g_1000','b_1000'],
            'sat2500': ['imtype','prop_lake_2500','r_2500','g_2500','b_2500'],
            'sat500_1000': ['imtype','prop_lake_500','r_500','g_500','b_500', 'prop_lake_1000','r_1000','g_1000','b_1000'],
            'sat500_2500': ['imtype','prop_lake_500','r_500','g_500','b_500', 'prop_lake_2500','r_2500','g_2500','b_2500'],
            'sat1000_2500': ['imtype','prop_lake_1000','r_1000','g_1000','b_1000', 'prop_lake_2500','r_2500','g_2500','b_2500']}

sat_keys = tuple(sat_cats.keys())


In [27]:
def objective_lgb(trial):
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 20, 600, 10),
        "max_depth": trial.suggest_int("max_depth", 2, 10),
        "ele_set": trial.suggest_categorical("ele_vars", ele_keys),
        "xy_set": trial.suggest_categorical("xy_set", xy_keys),
        "sl_set": trial.suggest_categorical("sl_set", sl_keys),
        "reg_set": trial.suggest_categorical("reg_set", reg_keys),
        "weight": trial.suggest_categorical("weight", weight_cats),
        "cat_type": trial.suggest_categorical("cat_type", cat_cats),
        "sat_set": trial.suggest_categorical("sat_set", sat_keys),
    }
    # Setting the different variables
    ov = region_cats[param['reg_set']]
    #if 'imtype' in sat_cats[param['sat_set']]:
    #    ov.append('imtype')
    cv = ele_cats[param['ele_set']] + xy_cats[param['xy_set']]
    cv += sl_cats[param['sl_set']]
    cv += sat_cats[param['sat_set']]
    rm = mod.RegMod(ord_vars=ov,
                    dum_vars=None,
                    dat_vars=['date'],
                    ide_vars=cv,
                    weight = 'split_pred',
                    y='severity',
                    mod = mod.LGBMRegressor(n_estimators=round(param['n_estimators']),
                                            max_depth=param['max_depth']))
    avg_rmse = rm.met_eval(train_dat,ret=True,weight=param['weight'],cat=param['cat_type'])
    return avg_rmse

study_lgb = optuna.create_study(direction="minimize")
study_lgb.optimize(objective_lgb, n_trials=300) # 150
trial_lgb = study_lgb.best_trial
res_results['lgb'] = trial_lgb

[I 2023-04-14 19:58:50,330] A new study created in memory with name: no-name-3260d55f-e0c4-47ca-937b-00d0582b473f
[I 2023-04-14 19:59:07,744] Trial 0 finished with value: 0.8079305712392552 and parameters: {'n_estimators': 230, 'max_depth': 4, 'ele_vars': 'ele_std', 'xy_set': 'lat', 'sl_set': 'lagNone', 'reg_set': 'clust', 'weight': False, 'cat_type': True, 'sat_set': 'sat500'}. Best is trial 0 with value: 0.8079305712392552.
[I 2023-04-14 19:59:30,775] Trial 1 finished with value: 0.8275873216645457 and parameters: {'n_estimators': 460, 'max_depth': 3, 'ele_vars': 'all_var', 'xy_set': 'no', 'sl_set': 'lagNone', 'reg_set': 'clust', 'weight': False, 'cat_type': True, 'sat_set': 'sat500_2500'}. Best is trial 0 with value: 0.8079305712392552.
[I 2023-04-14 20:00:06,758] Trial 2 finished with value: 0.8235443492503702 and parameters: {'n_estimators': 260, 'max_depth': 6, 'ele_vars': 'all_var', 'xy_set': 'no', 'sl_set': 'lagNone', 'reg_set': 'reg', 'weight': False, 'cat_type': True, 'sat_se

## Best Model

In [23]:
# src- main_preds.py
train_dat = get_data(1, split_pred=True)
train_dat.head()

# Example just predicting severity directly

sat_500 = ['prop_lake_500', 'r_500', 'g_500', 'b_500']
sat_1000 = ['prop_lake_1000', 'r_1000', 'g_1000', 'b_1000']
sat_2500 = ['prop_lake_2500', 'r_2500', 'g_2500', 'b_2500']
sat_1025 = ['prop_lake_2500', 'r_2500', 'g_2500', 'b_2500', 
           'prop_lake_1000', 'r_1000', 'g_1000', 'b_1000']

cat = mod.RegMod(ord_vars=['region','cluster'],
                dat_vars=['date'],
                ide_vars=['latitude','longitude','maxe','dife'],
                y='severity',
                mod = mod.CatBoostRegressor(iterations=380,depth=6,
                   allow_writing_files=False,verbose=False)
                )
cat.fit(train_dat,weight=False,cat=False)

In [13]:
lig = mod.RegMod(ord_vars=['region','cluster','imtype'],
                dat_vars=['date'],
                ide_vars=['latitude','longitude','elevation','dife'] + sat_1025,
                y='severity',
                mod = mod.LGBMRegressor(n_estimators=470,max_depth=8)
                )
lig.fit(train_dat,weight=False,cat=True)

In [14]:
xgb = mod.RegMod(ord_vars=['region','cluster'],
                 dat_vars=['date'],
                 y='severity',
                 mod = mod.XGBRegressor(n_estimators=70, max_depth=2))
xgb.fit(train_dat,weight=False,cat=False)

In [15]:
rm = mod.EnsMod(mods={'xgb': xgb, 'cat': cat, 'lig': lig})

In [16]:
# Now getting files for out of sample data
test = feat.get_data(data_type='test')
test['pred'] = rm.predict_int(test)

In [17]:
form_dat = feat.sub_format(test)
print(form_dat['severity'].value_counts())

# function to check if similar to any past submissions
mod.check_similar(form_dat)

# Checking to see differences compared to best submission so far
current = form_dat.copy()
mod.check_day(current,day="sub_2023_02_16.csv")
current.groupby('region',as_index=False)['dif_2023_02_16'].value_counts()

# Saving the data and model
form_dat.to_csv(f'sub_BESTRESULTS_0414.csv',index=False)
mod.save_model(rm,f'mod_BESTRESULTS_0414')

2    5149
1    1253
3     108
Name: severity, dtype: int64
 0    2914
 2    1836
 1     968
-1     561
 3     227
-2       4
Name: dif_2023_02_16, dtype: int64
